In [ ]:
!pip install -qU langchain_ollama
!pip install -qU "langchain-chroma>=0.1.2"

In [20]:
!pip install --quiet langchain_experimental langchain_openai

In [2]:
!!ollama pull all-minilm

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest 
pulling 797b70c4edf8...   0% â–•                â–�    0 B/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   0% â–•                â–�    0 B/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   0% â–•                â–�    0 B/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   0% â–•                â–�    0 B/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   0% â–•                â–� 127 KB/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   1% â–•                â–� 311 KB/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   1% â–•                â–� 647 KB/ 4

In [3]:
!ollama pull wizardlm2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ´ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest 
pulling 431800fe7a30...   3% â–•                â–� 142 MB/4.1 GB                  pulling manifest 
pulling 431800fe7a30...   3% â–•                â–� 142 MB/4.1 GB                  pulling manifest 
pulling 431800fe7a30...   3% â–•                â–� 143 MB/4.1 GB                  pulling manifest 
pulling 431800fe7a30...   3% â–•                â–� 143 MB/4.1 GB                  pulling manifest 
pulling 431800fe7a30...   4% â–•                â–� 144 MB/4.1 GB                  pulling manifest 
pulling 431800fe7a30...   4% â–•                â–� 144 MB/4.1 GB                  pulling manifest 
pulling 431800fe7a30...   4% â–•                â–� 144 MB/4.1 GB                  pulling manifest 
pulling 431800fe7a30...   4% â–•                â–� 145 MB/4.1 GB                  pul

In [2]:
%cd C:\Users\natha\OneDrive\Documents\University of Michigan\Capstone\

C:\Users\natha\OneDrive\Documents\University of Michigan\Capstone


In [4]:
from langchain_ollama import OllamaEmbeddings
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
import pandas as pd
import chromadb

embeddings_model = OllamaEmbeddingFunction(model_name="wizardlm2",  url="http://localhost:11434/api/embeddings")

In [5]:
# Run if need to create a new dataset
import chromadb
client = chromadb.PersistentClient(path="collection_test")

In [6]:
# Run if need to delete collection
client.delete_collection(name="trial_collection")

In [7]:
collection = client.create_collection(name="trial_collection", embedding_function=embeddings_model)

In [8]:
from bs4 import BeautifulSoup
from langchain_experimental.text_splitter import SemanticChunker
import os

def ticker_toChromaDB(ticker, collection,model):

    file = 'filings/' + ticker + '/' + os.listdir('filings/' + ticker + '/')[0]

    with open(file, 'r', encoding='utf-8') as file:
        html_content = file.read()
        soup = BeautifulSoup(html_content, 'html.parser')

    text_list = []
    for div in soup.find_all('div'):
        if not div.find('table'):
            text_list.append(div.get_text(separator="\n", strip=True))


    seen = set()
    result = ''
    for text in text_list[1:]:
        if text and len(text) > 20 and text not in seen:
            seen.add(text)
            result += text

    text_splitter = SemanticChunker(OllamaEmbeddings(model=model), breakpoint_threshold_type="gradient", breakpoint_threshold_amount = 92.5)


    docs = text_splitter.create_documents([result])
    print('splitter done, number of splits:' + str(len(docs)))

    collection.add(documents=[i.page_content for i in docs],
                   ids = [ticker + str(i) for i in range(len(docs))],
                   metadatas=
                   [
                       {"ticker": ticker}
                       for _ in docs
                   ]
                   )
    print('Collection Updated')


In [ ]:
for ticker in ['A','AAPL','ABBV']:
    print(ticker)
    ticker_toChromaDB(ticker, collection, 'wizardlm2')

A


In [36]:
collection.peek(20)

{'ids': ['id0',
  'id1',
  'id2',
  'id3',
  'id4',
  'id5',
  'id6',
  'id7',
  'id8',
  'id9',
  'id10',
  'id11',
  'id12',
  'id13',
  'id14',
  'id15',
  'id16',
  'id17',
  'id18',
  'id19'],
 'embeddings': array([[-0.0617009 , -0.01183392, -0.10122396, ..., -0.02932424,
          0.01412324,  0.0477089 ],
        [-0.11959186, -0.02669412, -0.00576165, ..., -0.17078665,
          0.10155325,  0.12747701],
        [-0.17355201,  0.02303651, -0.0163439 , ..., -0.07493553,
         -0.06082832,  0.04086946],
        ...,
        [-0.21661976, -0.00857204,  0.03912531, ..., -0.16720155,
          0.00575124, -0.10622647],
        [-0.11813144, -0.14050043,  0.03922797, ..., -0.33301625,
          0.00743607, -0.00576364],
        [-0.09415805,  0.0094487 , -0.1458946 , ..., -0.18406752,
          0.11871351,  0.1443546 ]]),
 'documents': ['SECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549☒\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934Fo

In [37]:
collection.query(query_texts=["What are the main properties?"], n_results=2)

{'ids': [['id5', 'id14']],
 'embeddings': None,
 'documents': [['| 2020 Form 10-K | 4Item 1A. Risk FactorsThe following discussion of risk factors contains forward-looking statements. These risk factors may be important to understanding other statements in this Form 10-K. The following information should be read in conjunction with Part II, Item 7, “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and the consolidated financial statements and accompanying notes in Part II, Item 8, “Financial Statements and Supplementary Data” of this Form 10-K.The business, financial condition and operating results of the Company can be affected by a number of factors, whether currently known or unknown, including but not limited to those described below, any one or more of which could, directly or indirectly, cause the Company’s actual financial condition and operating results to vary materially from past, or from anticipated future, financial condition and operat